In [1]:
import sys
if True:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    root = "/content/drive/My Drive/Colab Notebooks/burgers/"
    sys.path.append("/content/drive/My Drive/Colab Notebooks/burgers/")
else:
    root = './'
import numpy as np
import torch
from matplotlib import pylab as plt
if torch.cuda.device_count()>0:
    device = torch.device('cuda')
    print("Connected to a GPU")
else:
    print("Using the CPU")
    device = torch.device('cpu')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
Using the CPU


In [0]:
import importlib
import util
importlib.reload(util)
from util import *

In [0]:
loss = torch.nn.MSELoss()

In [0]:
dataname = "heat"
data = np.load(root+f"data/{dataname}.npz")
dataset = torch.tensor(data['U'],dtype=torch.float32,device=device)
Ntraj,Nt,Nx = dataset.shape

In [0]:
def train_it(dataset, model,
             OPTIM=torch.optim.Adam, learning_rate = 1e-1, 
             batch_size=20, Npast=1, Nfuture=1):
    optimizer = OPTIM(model.parameters(), lr=learning_rate)
    Nepoch = 100
    Ntotal = dataset.shape[0]*(dataset.shape[1]-(Npast+1)-Nfuture)
    Nbatch = batch_size #Ntotal//10
    Niter = Nepoch * Ntotal//Nbatch
    Nprint = (Niter)//10
    Nsave = 100
    losses = np.zeros((Niter//Nsave,2))
    
    for e in range(Niter):
        xx,yy = get_batch(Nbatch, dataset, Nfuture=Nfuture)
        y_pred = model(xx)+xx
        L = loss(y_pred,yy[:,(0,),:])
        h = y_pred
        #for i in range(Nfuture-1):
        optimizer.zero_grad()
        L.backward()
        optimizer.step()
        if e%Nprint == Nprint-1:
            print(e,L.item())
        if e%Nsave == Nsave-1:
            xxt,yyt = test_batch(dataset)
            yt_pred = model(xxt)+xxt
            test_err = loss(yyt,yt_pred)
            losses[e//Nsave,0] = L.item()
            losses[e//Nsave,1] = test_err.item()
    return losses

In [29]:
name = "PureStencil"
model_cls = models[name]
for nf in [1,]:
    model = model_cls(Nx).to(device)
    results = train_it(dataset,model,Nfuture=nf)

484 8.323387010022998e-05
969 2.0462965039769188e-05
1454 9.622901416150853e-05
1939 8.946689376898576e-06
2424 3.865981398121221e-06
2909 8.258575121544709e-07
3394 2.820351983245928e-05
3879 7.114944367003773e-08
4364 2.8788932127099542e-08
4849 7.759794584671909e-08


In [0]:
xx,yy = get_batch(1, dataset, Nfuture=3)

In [23]:
yy.shape

torch.Size([1, 3, 41])

In [24]:
xx.shape

torch.Size([1, 1, 41])

In [27]:
yy[:,(0,),:].shape

torch.Size([1, 1, 41])